In [1]:
import warnings
with warnings.catch_warnings():
    warnings.filterwarnings("ignore",category=FutureWarning)
    import h5py
import numpy as np
import math
import matplotlib.pyplot as plt  
import numba
try:
    profile  # throws an exception when profile isn't defined
except NameError:
    profile = lambda x: x

In [2]:
@numba.jit
def onemove_in_cube_true_numba(p0,v): 
    v[v==0]=1e-16
    htime=np.abs((np.floor(p0)-p0+(v>0))/v)
    minLoc=np.argmin(htime)
    dist=htime[minLoc]
    htime=p0+dist*v
    htime[minLoc]=round(htime[minLoc])+np.spacing(abs(htime[minLoc]))*np.sign(v[minLoc])
    return htime,dist

In [3]:
def main_loop(Mx,My,Nx,Ny,Nz,h,D,orginOffset,ep,mu):
    detector=np.zeros((Mx,My),dtype=np.float32)
    for z in range(0,Mx*My): 
        j=z%Mx
        i=int(z/Mx) 
        pos=np.array([orginOffset[0]+i*D,orginOffset[1]+D*j, 0],dtype=np.float32)
        dir=((ep-pos)/np.linalg.norm(ep-pos)).astype(np.float32)
        L=1
        while pos[2]< h+Nz:
            pos,dist=onemove_in_cube_true_numba(pos,dir)
            if 0 <= pos[0] < Nx and 0<=pos[1]<Ny  and h<=pos[2] < h+Nz:
                L=L*np.exp(-1*mu[math.floor(pos[0]),math.floor(pos[1]),math.floor(pos[2]-h)]*dist)
        detector[i][j] = L;
    return detector

In [4]:
Nx = 208
Ny = 256 
Nz = 225
Mx = 200
My = 200
D = 2
h = 50
H = h + Nz + 200
orginOffset = np.array(
        [(-Mx * D) / 2 + (Nx / 2), (-My * D) / 2 + (Ny / 2), 0], dtype=np.float32)
ep = np.array([Nx / 2, Ny / 2, H], dtype = np.float32)
muBone = 0.573
muFat = 0.193
orginOffset = np.array([(-Mx * D) / 2 + (Nx / 2), (-My * D) / 2 +(Ny / 2), 0],dtype=np.float32) 
ep = np.array([Nx / 2, Ny / 2, H],dtype=np.float32)                                              
f = h5py.File('headct.h5', 'r')
headct=np.array(f.get('ct'))
headct=np.transpose(headct)
det=f.get('det')
det=np.transpose(det)
mu=np.zeros((Nx,Ny,Nz),dtype=np.float32)
mu[np.nonzero(headct>0)]=((headct[np.nonzero(headct>0)]-0.3)/(0.7))*(muBone-muFat)+muFat

In [5]:
main_num=numba.jit(main_loop,nopython=True,debug=True)

In [6]:
detector=main_num(Mx,My,Nx,Ny,Nz,h,D,orginOffset,ep,mu)

In [ ]:
main_num.inspect_types()

In [7]:
print(np.isclose(det,detector,rtol=.5).all())
np.save('Detector.npy',detector)

True


In [ ]:
#Numpy
%timeit onemove_in_cube_true(np.array([1,2,2],dtype='float32'),np.array([.11,.22,.12],dtype='float32'))

In [ ]:
#cython
%timeit move_step.onemove_in_cube_true(np.array([1,2,2],dtype='float32'),np.array([.11,.22,.12],dtype='float32'))

In [ ]:
#Numba No Type
%timeit onemove_in_cube_true_numba(np.array([1,2,2],dtype='float32'),np.array([.11,.22,.12],dtype='float32'))

In [13]:
np.size(headct,0)

208